In [1]:
# importing necessary libraries
import tensorflow as tf
import transformers
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import csv
from transformers import BertTokenizer, TFBertModel

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [2]:
f = open('spotify_million_playlist_dataset_challenge/challenge_set.json')

js = json.load(f)
playlists = js['playlists']
titles = []
tracks = []

for playlist in playlists:
    if not playlist['tracks'] or 'name' not in playlist:
        continue
    titles.append(playlist['name'].lower())
    tracks.append(' '.join(track['track_uri'] for track in playlist['tracks']))
        

In [3]:
def tokenize(sentences):
    text_tokenizer = Tokenizer()
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [4]:
titles_tokens, title_tokenizer = tokenize(titles)
tracks_tokens, track_tokenizer = tokenize(tracks)

title_vocab = len(title_tokenizer.word_index) + 1
track_vocab = len(track_tokenizer.word_index) + 1

max_title_length = int(len(max(titles_tokens, key=len)))
max_track_length = int(len(max(tracks_tokens, key=len)))

pad_titles = pad_sequences(titles_tokens, max_title_length, padding = "post")
pad_tracks = pad_sequences(tracks_tokens, max_track_length, padding = "post")

pad_titles = pad_titles.reshape(*pad_titles.shape, 1)
pad_tracks = pad_tracks.reshape(*pad_tracks.shape, 1)

In [5]:
input_sequence = Input(shape=(max_title_length,))
embedding = Embedding(input_dim=title_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_track_length)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(track_vocab))(decoder)

In [6]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding (Embedding)       (None, 9, 128)            285440    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVecto  (None, 300, 64)           0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 300, 64)           33024     
                                                                 
 time_distributed (TimeDist  (None, 300, 63999)        4159935   
 ributed)                                                    

In [7]:
model_results = enc_dec_model.fit(pad_titles, pad_tracks, batch_size=30, epochs=1)

def logits_to_sentence(logits, tokenizer):
    print(logits)
    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print(logits_to_sentence(enc_dec_model.predict(pad_titles[14:15])[0], track_tokenizer))

2023-12-04 15:31:51.756763: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp_10.


1/1 [==============================] - 1s 841ms/step
[[3.41761410e-01 1.10834539e-01 1.11249350e-01 ... 7.53076534e-09
  1.12409859e-08 1.50910999e-08]
 [6.27935648e-01 1.47710741e-01 1.48440674e-01 ... 9.03791555e-11
  1.52589691e-10 2.22744545e-10]
 [6.52597785e-01 1.45933568e-01 1.46682143e-01 ... 4.25743642e-11
  7.32856067e-11 1.08406034e-10]
 ...
 [6.56482816e-01 1.45515367e-01 1.46264166e-01 ... 3.73279532e-11
  6.44715809e-11 9.55886342e-11]
 [6.56482816e-01 1.45515367e-01 1.46264166e-01 ... 3.73279532e-11
  6.44715809e-11 9.55886342e-11]
 [6.56482816e-01 1.45515367e-01 1.46264166e-01 ... 3.73279532e-11
  6.44715809e-11 9.55886342e-11]]
<empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <em